# フェーズ1 - 2週目：2量子ビットシミュレータの実装

このノートブックでは、シミュレータを2量子ビットに拡張します。複数の量子ビットを扱うための数学的な道具である**テンソル積**を学び、2量子ビットに作用する**CNOTゲート**を実装して、有名な\*\*ベル状態（量子もつれ）\*\*を生成します。

## 1\. ライブラリと基本定義

前回同様、NumPyをインポートし、1量子ビットの状態とゲートを定義します。今回は何もしないことを表す**I (Identity)ゲート**も追加します。

In [1]:
import numpy as np

# --- 1量子ビットの定義 ---
# 状態ベクトル
qbit_0 = np.array([1+0j, 0+0j])
qbit_1 = np.array([0+0j, 1+0j])

# 1量子ビットゲート
gate_i = np.array([[1, 0], [0, 1]]) # I (Identity) ゲート
gate_h = np.array([[1, 1], [1, -1]]) / np.sqrt(2) # H (Hadamard) ゲート

## 2\. 複数量子ビットの表現：テンソル積 ⊗

2つの量子ビットの状態を1つのベクトルで表現するには、**テンソル積**（クロネッカー積）という計算を使います。NumPyでは `np.kron()` で簡単に計算できます。

例えば、2つの量子ビットが両方とも$|0\rangle$である状態 $|00\rangle$ は、数学的には $|0\rangle \otimes |0\rangle$ と書かれ、以下のように計算されます。

$$|00\rangle = |0\rangle \otimes |0\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \otimes \begin{pmatrix} 1 \\ 0 \end{pmatrix} = \begin{pmatrix} 1 \times \begin{pmatrix} 1 \\ 0 \end{pmatrix} \\ 0 \times \begin{pmatrix} 1 \\ 0 \end{pmatrix} \end{pmatrix} = \begin{pmatrix} 1 \\ 0 \\ 0 \\ 0 \end{pmatrix}$$

これにより、2量子ビットの状態は4次元のベクトルで表現されます。

  - $|00\rangle \rightarrow [1, 0, 0, 0]^T$
  - $|01\rangle \rightarrow [0, 1, 0, 0]^T$
  - $|10\rangle \rightarrow [0, 0, 1, 0]^T$
  - $|11\rangle \rightarrow [0, 0, 0, 1]^T$

In [2]:
# |0> と |0> のテンソル積で |00> を作る
state_00 = np.kron(qbit_0, qbit_0)
print(f"|00> = {state_00}")

# |1> と |0> のテンソル積で |10> を作る
state_10 = np.kron(qbit_1, qbit_0)
print(f"|10> = {state_10}")

|00> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
|10> = [0.+0.j 0.+0.j 1.+0.j 0.+0.j]


## 3\. 2量子ビットゲートの実装

### CNOT (Controlled-NOT) ゲート

CNOTゲートは、**コントロールビット**と**ターゲットビット**の2つを取ります。

  - コントロールビットが$|0\rangle$なら、ターゲットビットは何もしない。
  - コントロールビットが$|1\rangle$なら、ターゲットビットを反転（Xゲートを適用）する。

1番目の量子ビットをコントロール、2番目をターゲットとした場合の行列は以下のようになります。
$$CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}$$

### 特定の量子ビットへのゲート適用

1番目の量子ビットにのみHゲートをかけ、2番目には何もしない（Iゲートをかける）場合、システム全体に作用するゲートは $H \otimes I$ のようにテンソル積で表現します。

In [3]:
# CNOTゲートの定義 (1番目がコントロール)
gate_cnot = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

# 1番目の量子ビットにH、2番目にIを作用させるゲート
gate_h_on_q1 = np.kron(gate_h, gate_i)

print("--- CNOT Gate ---")
print(gate_cnot)
print("\n--- H on q1, I on q2 ---")
print(gate_h_on_q1)

--- CNOT Gate ---
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]]

--- H on q1, I on q2 ---
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678 -0.        ]
 [ 0.          0.70710678 -0.         -0.70710678]]


## 4\. シミュレーション：ベル状態の生成 🌌

いよいよ、量子もつれ状態である**ベル状態 $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$** を作ります。
これは以下の手順で生成できます。

1.  初期状態として $|00\rangle$ を用意する。
2.  1番目の量子ビットにHゲートを適用する。（システム全体では $H \otimes I$ を適用）
3.  1番目をコントロール、2番目をターゲットとしてCNOTゲートを適用する。


In [4]:
# 1. 初期状態 |00> を用意
initial_state = np.kron(qbit_0, qbit_0)
print(f"初期状態 |00>: \n{initial_state}\n")

# 2. 1番目の量子ビットにHゲートを適用
state_after_h = np.dot(gate_h_on_q1, initial_state)
print(f"Hゲート適用後 (1/√2 * (|00> + |10>)) : \n{state_after_h}\n")

# 3. CNOTゲートを適用
final_state = np.dot(gate_cnot, state_after_h)
print(f"最終状態 (ベル状態): \n{final_state}\n")

# 検算
# 期待されるベクトルは [1/√2, 0, 0, 1/√2]
print(f"期待される振幅: {[1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]}")

初期状態 |00>: 
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]

Hゲート適用後 (1/√2 * (|00> + |10>)) : 
[0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]

最終状態 (ベル状態): 
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]

期待される振幅: [np.float64(0.7071067811865475), 0, 0, np.float64(0.7071067811865475)]


最終結果のベクトル `[0.707..., 0., 0., 0.707...]` は、$|00\rangle$ の振幅と $|11\rangle$ の振幅がそれぞれ $1/\sqrt{2}$ であり、他は0であることを示しています。これはまさにベル状態です！

この状態は、測定すると50%の確率で`00`が、50%の確率で`11`が観測され、`01`や`10`は決して観測されないという、古典的にはありえない相関（量子もつれ）を持っています。

これで2週目の目標は達成です。次はn量子ビットへの拡張と測定機能の実装に進みましょう。